# Creating 2 Tables: Sites for Map, Allocations for Table

#### General Approach:
1) Read in csv sheet of data.  Export fields into new database.
2) Remove bad rows and elements.
3) Fix date datatype.
4) Include ID int value for beneficial use.
5) Fix and remove misisng or nan values for allocatoin flow and volume.
6) Remove duplicates based on AllocationNativeID.
7) Create API link for each siteUUID.
8) Export completed df as processed csv.

### Sites LF Allow, for creating Site Points, Popups and Legend Colors on Map Only

In [1]:
#Needed Libaraies
import os
import numpy as np
import pandas as pd

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [2]:
#Sites LeftJoin Allo Input
SitefileInput = "dontopen_SiteLJAllow.csv"
dfsite = pd.read_csv(SitefileInput)
dfsite

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SiteID,SiteUUID,Longitude,Latitude,AllocationNativeID,AllocationAmountID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,BeneficialUseCV,Date,WaterSourceID,WaterSourceName,OrganizationID,OrganizationUUID,State
0,278477,test,NaN,NaN,NaN,1.0,NaN,NaN,NaN,9.0,Irrigation,Irrigation,2019-04-25,373069.0,test,4.0,test,UT
1,3438539,WA_1,-119.798724,46.583691,2.08574e+06,1384605.0,4.790220,0.00,175.0,16932.0,Irrigation,Irrigation,1990-06-06,462900.0,Unspecified,11.0,WSDE,WA
2,3438539,WA_1,-119.798724,46.583691,4.27133e+06,1438026.0,2.562211,345.26,283.0,9445.0,Irrigation,Irrigation,2006-04-27,462900.0,Unspecified,11.0,WSDE,WA
3,3438539,WA_1,-119.798724,46.583691,4.2716e+06,1438029.0,3.342014,345.26,283.0,9445.0,Irrigation,Irrigation,2006-04-27,462900.0,Unspecified,11.0,WSDE,WA
4,3438540,WA_10,-120.336077,46.486242,4.54626e+06,1439033.0,0.891204,7.00,0.0,34607.0,Irrigation,NaN,2008-10-03,462900.0,Unspecified,11.0,WSDE,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118741,4407198,TX_9995,-97.136661,28.907579,P3853,2139064.0,NaN,NaN,NaN,44037.0,Irrigation,NaN,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX
1118742,4407199,TX_9996,-97.137044,28.900439,P3853,2139064.0,NaN,NaN,NaN,44037.0,Irrigation,NaN,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX
1118743,4407200,TX_9997,-97.136423,28.893273,C3859,2141489.0,NaN,NaN,NaN,44037.0,Irrigation,NaN,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX
1118744,4407201,TX_9998,-97.131844,28.883523,C5486,2138971.0,NaN,NaN,NaN,44037.0,Irrigation,Industrial,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX


In [3]:
#removeing bad rows of df because they are null for some odd reason
dfsite = dfsite[(dfsite.WaterSourceName != 'test')]
dfsite = dfsite[dfsite['SiteUUID'].notna()]
dfsite = dfsite[dfsite['AllocationPriorityDateID'].notna()]
dfsite = dfsite[dfsite['BeneficialUseCV'].notna()]
dfsite = dfsite[dfsite['Longitude'].notna()]
dfsite = dfsite[dfsite['Latitude'].notna()]
dfsite.reset_index()

,index,SiteID,SiteUUID,Longitude,Latitude,AllocationNativeID,AllocationAmountID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,BeneficialUseCV,Date,WaterSourceID,WaterSourceName,OrganizationID,OrganizationUUID,State
0,1,3438539,WA_1,-119.798724,46.583691,2.08574e+06,1384605.0,4.790220,0.00,175.0,16932.0,Irrigation,Irrigation,1990-06-06,462900.0,Unspecified,11.0,WSDE,WA
1,2,3438539,WA_1,-119.798724,46.583691,4.27133e+06,1438026.0,2.562211,345.26,283.0,9445.0,Irrigation,Irrigation,2006-04-27,462900.0,Unspecified,11.0,WSDE,WA
2,3,3438539,WA_1,-119.798724,46.583691,4.2716e+06,1438029.0,3.342014,345.26,283.0,9445.0,Irrigation,Irrigation,2006-04-27,462900.0,Unspecified,11.0,WSDE,WA
3,5,3438540,WA_10,-120.336077,46.486242,4.54603e+06,1439032.0,2.228009,774.00,181.0,34607.0,Irrigation,Irrigation,2008-10-03,462900.0,Unspecified,11.0,WSDE,WA
4,13,3438548,WA_10005,-120.230821,45.984583,2.04948e+06,1374895.0,0.066840,12.50,2.0,1425.0,Irrigation,Irrigation,1974-06-01,462900.0,Unspecified,11.0,WSDE,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766354,1118691,4407148,TX_995,-96.015061,30.135926,C5318,2141496.0,NaN,NaN,NaN,44037.0,Irrigation,Recreation,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX
766355,1118702,4407159,TX_996,-96.644058,30.822718,P4145,2138007.0,NaN,NaN,NaN,44037.0,Irrigation,Irrigation,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX
766356,1118713,4407170,TX_997,-101.479177,33.471532,C3708,2141254.0,NaN,NaN,NaN,44037.0,Irrigation,Irrigation,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX
766357,1118724,4407181,TX_998,-101.479493,33.471399,C3708,2141254.0,NaN,NaN,NaN,44037.0,Irrigation,Irrigation,2020-06-24,559475.0,Unspecified,26.0,TCEQ,TX


In [4]:
SiteWSWCBenUseDict = {
"Irrigation" : "Agricultural",
"Agriculture" : "Agricultural",
"Irrigation Storage" : "Agricultural",
"Agriculture other than irrigation" : "Agricultural",
"Commercial" : "Commercial",
"Commercial From Storage" : "Commercial",
"Commercial Storage" : "Commercial",
"72-12-1 Sanitary in conjunction with a commercial use" : "Commercial",
"Domestic" : "Domestic",
"72-12-1 domestic and livestock watering" : "Domestic",
"72-12-1 domestic one household" : "Domestic",
"72-12-1 multiple domestic households" : "Domestic",
"Mobile home parks" : "Domestic",
"Subdivision" : "Domestic",
"Household Use Only" : "Domestic",
"Domestic From Storage" : "Domestic",
"Domestic Storage" : "Domestic",
"Public utility" : "Domestic",
"Minimum Streamflow" : "Environmental",
"Ground Water Recharge" : "Environmental",
"Ground Water Recharge From Storage" : "Environmental",
"Ground Water Recharge Storage" : "Environmental",
"Lake Level Maintenance" : "Environmental",
"Minimum Stream Flow" : "Environmental",
"Mitigation" : "Environmental",
"72-12-1 Prospecting or development of natural resource" : "Environmental",
"Pollution control well" : "Environmental",
"Pollution Abatement" : "Environmental",
"Fire" : "Fire",
"Fire Protection" : "Fire",
"Fire Protection From Storage" : "Fire",
"Fire Protection Storage" : "Fire",
"Flood Control Storage" : "Flood Control",
"Flood Control" : "Flood Control",
"Industrial" : "Industrial",
"Cemetery" : "Industrial",
"Construction" : "Industrial",
"Dairy operation" : "Industrial",
"Domestic construction" : "Industrial",
"Highway construction" : "Industrial",
"Oil production" : "Industrial",
"Poultry and egg operation" : "Industrial",
"Industrial From Storage" : "Industrial",
"Industrial Storage" : "Industrial",
"Irrigation From Storage" : "Industrial",
"72-12-1 Construction of public works" : "Industrial",
"Petroleum processing plant" : "Industrial",
"Secondary recovery of oil" : "Industrial",
"Stockwater" : "Livestock",
"Stockwater From Storage" : "Livestock",
"Stockwater Storage" : "Livestock",
"72-12-1 livestock watering" : "Livestock",
"Stock" : "Livestock",
"Stockwatering" : "Livestock",
"Mining" : "Mining",
"Mining From Storage" : "Mining",
"Mining Storage" : "Mining",
"Mining or milling or oil" : "Mining",
"Municipal" : "Municipal",
"Municipal From Storage" : "Municipal",
"Municipal Storage" : "Municipal",
"Municipal - city or county supplied water" : "Municipal",
"Geothermal" : "Power",
"Power From Storage" : "Power",
"Power Storage" : "Power",
"Geothermal boreholes" : "Power",
"Power" : "Power",
"Power Generation" : "Power",
"Recharge" : "Recharge",
"Recreation From Storage" : "Recreation",
"Recreation Storage" : "Recreation",
"Recreation" : "Recreation",
"Snow Making" : "Snow Making",
"Storage" : "Storage",
"Fishery" : "Wildlife",
"Fish Habitat" : "Wildlife",
"Fish Habitat Storage" : "Wildlife",
"Fish Propagation" : "Wildlife",
"Fish Propagation From Storage" : "Wildlife",
"Fish Propagation Storage" : "Wildlife",
"Wildlife From Storage" : "Wildlife",
"Wildlife Storage" : "Wildlife",
"Wildlife" : "Wildlife",
"Fish And Wildlife" : "Wildlife",
"Fish and game propogation" : "Wildlife"}


def assignWSWCBU(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return np.nan
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = SiteWSWCBenUseDict[String1]
        except:
            outList = "State Specific"
    return outList

dfsite['WSWCBeneficialUse'] = dfsite.apply(lambda row: assignWSWCBU(row['BeneficialUseCV']), axis=1)

In [5]:
#Site Map Color per point
#Mapdeck uses a build color pacakge called 'colourvalues library', which does use HEX color code, but has to end in an "FF".
WSWCBenUseColorDict = { 
"Agricultural" : "#FFFF00FF",
"Commercial" : "#008000FF",
"Domestic" : "#0000FFFF",
"Environmental" : "#32CD32FF",
"Fire" : "#FF0000FF",
"Flood Control" : "#00FFFFFF",
"Industrial" : "#800080FF",
"Livestock" : "#FFD700FF",
"Mining" : "#A52A2AFF",
"Municipal" : "#4B0082FF",
"Power" : "#FFA500FF",
"Recharge" : "#D2691EFF",
"Recreation" : "#FFC0CBFF",
"Snow Making" : "#F0FFF0FF",
"Storage" : "#F5DEB3FF",
"Wildlife" : "#ADFF2FFF",
"State Specific" : "#808080FF"}
 
def assignWSWCBUColor(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return "#708090"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSWCBenUseColorDict[String1]
        except:
            outList = "#708090"
    return outList

dfsite['WBenUseColor'] = dfsite.apply(lambda row: assignWSWCBUColor(row['WSWCBeneficialUse']), axis=1)

In [6]:
#Sort by SiteUUID and BenUse (this way Agriculture should show up first in the below list)
dfsite = dfsite.sort_values(by=['SiteUUID', 'WSWCBeneficialUse'], ascending=True).reset_index()
dfsite

,index,SiteID,SiteUUID,Longitude,Latitude,AllocationNativeID,AllocationAmountID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,...,PrimaryUseCategoryCV,BeneficialUseCV,Date,WaterSourceID,WaterSourceName,OrganizationID,OrganizationUUID,State,WSWCBeneficialUse,WBenUseColor
0,198836,3625678,CA_1,-124.100000,40.963100,A010767,1506149.0,NaN,0.0,0.0,...,Irrigation,Domestic,1944-02-14,546831.0,UNSP,17.0,CSWRCB,CA,Domestic,#0000FFFF
1,198837,3625679,CA_10,-121.173807,39.875367,A014760,1507757.0,NaN,0.0,0.0,...,Irrigation,Domestic,1952-04-17,546831.0,UNSP,17.0,CSWRCB,CA,Domestic,#0000FFFF
2,198838,3625680,CA_1000,-122.716594,41.730083,A021986,1511835.0,NaN,0.0,0.0,...,Irrigation,Domestic,1964-12-01,546829.0,UNSP (2),17.0,CSWRCB,CA,Domestic,#0000FFFF
3,198839,3625681,CA_10000,-120.647606,40.636866,C000897,1518296.0,NaN,0.0,0.0,...,Irrigation,Stockwatering,1977-09-07,546844.0,UNST,17.0,CSWRCB,CA,Livestock,#FFD700FF
4,198840,3625682,CA_10001,-120.895545,38.450445,C000934,1518333.0,NaN,0.0,0.0,...,Irrigation,Stockwatering,1977-09-21,546844.0,UNST,17.0,CSWRCB,CA,Livestock,#FFD700FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766354,32735,3463486,WA_9951,-120.305427,45.995916,2.04837e+06,1374471.0,0.066840,10.0,1.0,...,Irrigation,Irrigation,1945-01-01,462900.0,Unspecified,11.0,WSDE,WA,Agricultural,#FFFF00FF
766355,32740,3463491,WA_9956,-120.300857,45.995265,2.04947e+06,1374892.0,0.000000,0.0,0.0,...,Irrigation,Irrigation,1974-06-21,462900.0,Unspecified,11.0,WSDE,WA,Agricultural,#FFFF00FF
766356,32744,3463495,WA_996,-120.191545,48.529318,2.04725e+06,1373975.0,0.289641,35.0,15.0,...,Irrigation,Irrigation,1969-04-01,462900.0,Unspecified,11.0,WSDE,WA,Agricultural,#FFFF00FF
766357,32747,3463498,WA_9962,-120.305913,45.998268,2.04032e+06,1371119.0,0.000000,0.0,0.0,...,Irrigation,Irrigation,1974-06-30,462900.0,Unspecified,11.0,WSDE,WA,Agricultural,#FFFF00FF


In [7]:
## Create Sites Dataframe - with attached Allo and Benuse info ##
########################################
outdfSite = pd.DataFrame()

#The Columns
outdfSite['SiteUUID'] = dfsite['SiteUUID'].astype(str) 
outdfSite['Lat'] = dfsite['Latitude'].astype(float) 
outdfSite['Long'] = dfsite['Longitude'].astype(float)
outdfSite['WBenUse'] = dfsite['WSWCBeneficialUse'].astype(str)
outdfSite['WBenUseColor'] = dfsite['WBenUseColor'].astype(str)

#Drop Duplicates - SiteUUID
outdfSite = outdfSite.drop_duplicates(['SiteUUID'], keep="first").reset_index(drop=True)

outdfSite

,SiteUUID,Lat,Long,WBenUse,WBenUseColor
0,CA_1,40.963100,-124.100000,Domestic,#0000FFFF
1,CA_10,39.875367,-121.173807,Domestic,#0000FFFF
2,CA_1000,41.730083,-122.716594,Domestic,#0000FFFF
3,CA_10000,40.636866,-120.647606,Livestock,#FFD700FF
4,CA_10001,38.450445,-120.895545,Livestock,#FFD700FF
...,...,...,...,...,...
589296,WA_9951,45.995916,-120.305427,Agricultural,#FFFF00FF
589297,WA_9956,45.995265,-120.300857,Agricultural,#FFFF00FF
589298,WA_996,48.529318,-120.191545,Agricultural,#FFFF00FF
589299,WA_9962,45.998268,-120.305913,Agricultural,#FFFF00FF


In [8]:
#Exporting Outbound DataFrame to working csv file.
outdfSite.to_csv('P_SiteLFAllo.csv', index=False)  # The output

### Allo LF Site, for filtering and Table Return Only

In [9]:
#Needed Libaraies
import os
import numpy as np
import pandas as pd

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [10]:
#Allo LeftJoin Sites Input
AllofileInput = "dontopen_AlloLFSite.csv"
dfallo = pd.read_csv(AllofileInput)
dfallo

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,AllocationNativeID,AllocationAmountID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,BeneficialUseCV,Date,SiteID,SiteUUID,Longitude,Latitude,WaterSourceID,WaterSourceName,OrganizationID,OrganizationUUID,State
0,NaN,1,NaN,NaN,NaN,9,Irrigation,Irrigation,2019-04-25,278477,test,NaN,NaN,373069,test,4,test,UT
1,2.03218e+06,1370234,0.000045,2.0000,2.00,6403,Irrigation,Irrigation,1940-01-01,3464461,WA_25923,-122.781905,48.655508,462900,Unspecified,11,WSDE,WA
2,2.03221e+06,1370235,0.026736,1.0741,0.63,25059,Irrigation,Irrigation,1914-01-01,3481376,WA_42838,-122.309847,48.474909,462900,Unspecified,11,WSDE,WA
3,2.03221e+06,1370236,0.044560,1.0741,10.92,11107,Irrigation,Irrigation,1895-07-01,3477093,WA_38555,-122.227602,48.533434,462900,Unspecified,11,WSDE,WA
4,2.03233e+06,1370237,2.673611,0.0000,150.00,33863,Irrigation,Irrigation,1999-06-04,3462628,WA_9179,-121.007557,45.674734,462900,Unspecified,11,WSDE,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914184,C3439,2141586,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4399149,TX_275,-95.861930,28.845659,559475,Unspecified,26,TCEQ,TX
914185,C1861,2141587,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4400788,TX_4224,-98.918007,31.154508,559475,Unspecified,26,TCEQ,TX
914186,C3786,2141588,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4400806,TX_4240,-101.176992,35.887029,559475,Unspecified,26,TCEQ,TX
914187,C3786,2141588,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4400807,TX_4241,-101.174643,35.887459,559475,Unspecified,26,TCEQ,TX


In [11]:
#removeing bad rows of df because they are null for some odd reason
dfallo = dfallo[(dfallo.WaterSourceName != 'test')]
dfallo = dfallo[dfallo['SiteUUID'].notna()]
dfallo = dfallo[dfallo['AllocationPriorityDateID'].notna()]
dfallo = dfallo[dfallo['BeneficialUseCV'].notna()]
dfallo = dfallo[dfallo['Longitude'].notna()]
dfallo = dfallo[dfallo['Latitude'].notna()]
dfallo.reset_index()

,index,AllocationNativeID,AllocationAmountID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,BeneficialUseCV,Date,SiteID,SiteUUID,Longitude,Latitude,WaterSourceID,WaterSourceName,OrganizationID,OrganizationUUID,State
0,1,2.03218e+06,1370234,0.000045,2.0000,2.00,6403,Irrigation,Irrigation,1940-01-01,3464461,WA_25923,-122.781905,48.655508,462900,Unspecified,11,WSDE,WA
1,2,2.03221e+06,1370235,0.026736,1.0741,0.63,25059,Irrigation,Irrigation,1914-01-01,3481376,WA_42838,-122.309847,48.474909,462900,Unspecified,11,WSDE,WA
2,3,2.03221e+06,1370236,0.044560,1.0741,10.92,11107,Irrigation,Irrigation,1895-07-01,3477093,WA_38555,-122.227602,48.533434,462900,Unspecified,11,WSDE,WA
3,4,2.03233e+06,1370237,2.673611,0.0000,150.00,33863,Irrigation,Irrigation,1999-06-04,3462628,WA_9179,-121.007557,45.674734,462900,Unspecified,11,WSDE,WA
4,5,2.03234e+06,1370238,0.735243,0.0000,20.00,31963,Irrigation,Irrigation,1999-07-12,3480835,WA_42297,-119.516129,48.583960,462900,Unspecified,11,WSDE,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766354,914183,C1497,2141585,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4399944,TX_3465,-100.023706,30.489069,559475,Unspecified,26,TCEQ,TX
766355,914184,C3439,2141586,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4399149,TX_275,-95.861930,28.845659,559475,Unspecified,26,TCEQ,TX
766356,914185,C1861,2141587,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4400788,TX_4224,-98.918007,31.154508,559475,Unspecified,26,TCEQ,TX
766357,914186,C3786,2141588,NaN,NaN,NaN,44037,Irrigation,Irrigation,2020-06-24,4400806,TX_4240,-101.176992,35.887029,559475,Unspecified,26,TCEQ,TX


In [12]:
#changing data type of 'Date' to datatype, then changing format of 'Date' to yyyy-mm-dd
dfallo['Date'] = pd.to_datetime(dfallo['Date'], errors = 'coerce')
dfallo['Date'] = pd.to_datetime(dfallo["Date"].dt.strftime('%m/%d/%Ym'))

In [13]:
#Removing NaN, and missing (999) values from AllocaitonAmount, AllocaitonMaxium, IrrigatedAcerage.
dfallo['AllocationAmount'] = dfallo['AllocationAmount'].fillna(0)
dfallo['AllocationAmount'] = dfallo['AllocationAmount'].replace(999, 0)

dfallo['AllocationMaximum'] = dfallo['AllocationMaximum'].fillna(0)
dfallo['AllocationMaximum'] = dfallo['AllocationMaximum'].replace(999, 0)

dfallo['IrrigatedAcreage'] = dfallo['IrrigatedAcreage'].fillna(0)
dfallo['IrrigatedAcreage'] = dfallo['IrrigatedAcreage'].replace(999, 0)

In [14]:
alloWSWCBenUseDict = {
"Irrigation" : "Agricultural",
"Agriculture" : "Agricultural",
"Irrigation Storage" : "Agricultural",
"Agriculture other than irrigation" : "Agricultural",
"Commercial" : "Commercial",
"Commercial From Storage" : "Commercial",
"Commercial Storage" : "Commercial",
"72-12-1 Sanitary in conjunction with a commercial use" : "Commercial",
"Domestic" : "Domestic",
"72-12-1 domestic and livestock watering" : "Domestic",
"72-12-1 domestic one household" : "Domestic",
"72-12-1 multiple domestic households" : "Domestic",
"Mobile home parks" : "Domestic",
"Subdivision" : "Domestic",
"Household Use Only" : "Domestic",
"Domestic From Storage" : "Domestic",
"Domestic Storage" : "Domestic",
"Public utility" : "Domestic",
"Minimum Streamflow" : "Environmental",
"Ground Water Recharge" : "Environmental",
"Ground Water Recharge From Storage" : "Environmental",
"Ground Water Recharge Storage" : "Environmental",
"Lake Level Maintenance" : "Environmental",
"Minimum Stream Flow" : "Environmental",
"Mitigation" : "Environmental",
"72-12-1 Prospecting or development of natural resource" : "Environmental",
"Pollution control well" : "Environmental",
"Pollution Abatement" : "Environmental",
"Fire" : "Fire",
"Fire Protection" : "Fire",
"Fire Protection From Storage" : "Fire",
"Fire Protection Storage" : "Fire",
"Flood Control Storage" : "Flood Control",
"Flood Control" : "Flood Control",
"Industrial" : "Industrial",
"Cemetery" : "Industrial",
"Construction" : "Industrial",
"Dairy operation" : "Industrial",
"Domestic construction" : "Industrial",
"Highway construction" : "Industrial",
"Oil production" : "Industrial",
"Poultry and egg operation" : "Industrial",
"Industrial From Storage" : "Industrial",
"Industrial Storage" : "Industrial",
"Irrigation From Storage" : "Industrial",
"72-12-1 Construction of public works" : "Industrial",
"Petroleum processing plant" : "Industrial",
"Secondary recovery of oil" : "Industrial",
"Stockwater" : "Livestock",
"Stockwater From Storage" : "Livestock",
"Stockwater Storage" : "Livestock",
"72-12-1 livestock watering" : "Livestock",
"Stock" : "Livestock",
"Stockwatering" : "Livestock",
"Mining" : "Mining",
"Mining From Storage" : "Mining",
"Mining Storage" : "Mining",
"Mining or milling or oil" : "Mining",
"Municipal" : "Municipal",
"Municipal From Storage" : "Municipal",
"Municipal Storage" : "Municipal",
"Municipal - city or county supplied water" : "Municipal",
"Geothermal" : "Power",
"Power From Storage" : "Power",
"Power Storage" : "Power",
"Geothermal boreholes" : "Power",
"Power" : "Power",
"Power Generation" : "Power",
"Recharge" : "Recharge",
"Recreation From Storage" : "Recreation",
"Recreation Storage" : "Recreation",
"Recreation" : "Recreation",
"Snow Making" : "Snow Making",
"Storage" : "Storage",
"Fishery" : "Wildlife",
"Fish Habitat" : "Wildlife",
"Fish Habitat Storage" : "Wildlife",
"Fish Propagation" : "Wildlife",
"Fish Propagation From Storage" : "Wildlife",
"Fish Propagation Storage" : "Wildlife",
"Wildlife From Storage" : "Wildlife",
"Wildlife Storage" : "Wildlife",
"Wildlife" : "Wildlife",
"Fish And Wildlife" : "Wildlife",
"Fish and game propogation" : "Wildlife"}


def assignWSWCBU(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return np.nan
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = alloWSWCBenUseDict[String1]
        except:
            outList = "State Specific"
    return outList

dfallo['WSWCBeneficialUse'] = dfallo.apply(lambda row: assignWSWCBU(row['BeneficialUseCV']), axis=1)

In [15]:
#Sort by AllocationAmountID and SiteUUID
dfallo = dfallo.sort_values(by=['AllocationAmountID', 'SiteUUID'], ascending=True).reset_index()
dfallo

,index,AllocationNativeID,AllocationAmountID,AllocationAmount,AllocationMaximum,IrrigatedAcreage,AllocationPriorityDateID,PrimaryUseCategoryCV,BeneficialUseCV,Date,SiteID,SiteUUID,Longitude,Latitude,WaterSourceID,WaterSourceName,OrganizationID,OrganizationUUID,State,WSWCBeneficialUse
0,1,2.03218e+06,1370234,0.000045,2.0000,2.00,6403,Irrigation,Irrigation,1940-01-01,3464461,WA_25923,-122.781905,48.655508,462900,Unspecified,11,WSDE,WA,Agricultural
1,2,2.03221e+06,1370235,0.026736,1.0741,0.63,25059,Irrigation,Irrigation,1914-01-01,3481376,WA_42838,-122.309847,48.474909,462900,Unspecified,11,WSDE,WA,Agricultural
2,3,2.03221e+06,1370236,0.044560,1.0741,10.92,11107,Irrigation,Irrigation,1895-07-01,3477093,WA_38555,-122.227602,48.533434,462900,Unspecified,11,WSDE,WA,Agricultural
3,4,2.03233e+06,1370237,2.673611,0.0000,150.00,33863,Irrigation,Irrigation,1999-06-04,3462628,WA_9179,-121.007557,45.674734,462900,Unspecified,11,WSDE,WA,Agricultural
4,5,2.03234e+06,1370238,0.735243,0.0000,20.00,31963,Irrigation,Irrigation,1999-07-12,3480835,WA_42297,-119.516129,48.583960,462900,Unspecified,11,WSDE,WA,Agricultural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766354,914183,C1497,2141585,0.000000,0.0000,0.00,44037,Irrigation,Irrigation,2020-06-24,4399944,TX_3465,-100.023706,30.489069,559475,Unspecified,26,TCEQ,TX,Agricultural
766355,914184,C3439,2141586,0.000000,0.0000,0.00,44037,Irrigation,Irrigation,2020-06-24,4399149,TX_275,-95.861930,28.845659,559475,Unspecified,26,TCEQ,TX,Agricultural
766356,914185,C1861,2141587,0.000000,0.0000,0.00,44037,Irrigation,Irrigation,2020-06-24,4400788,TX_4224,-98.918007,31.154508,559475,Unspecified,26,TCEQ,TX,Agricultural
766357,914186,C3786,2141588,0.000000,0.0000,0.00,44037,Irrigation,Irrigation,2020-06-24,4400806,TX_4240,-101.176992,35.887029,559475,Unspecified,26,TCEQ,TX,Agricultural


In [16]:
## Create Sites Dataframe - with attached Allo and Benuse info ##
########################################
outdfallo = pd.DataFrame()

#The Columns
outdfallo['AA_ID'] = dfallo['AllocationAmountID'].astype(str) 
outdfallo['SiteUUID'] = dfallo['SiteUUID'].astype(str)
outdfallo['SN_ID'] = dfallo['AllocationNativeID'].astype(str) 
outdfallo['Lat'] = dfallo['Latitude'].astype(float) 
outdfallo['Long'] = dfallo['Longitude'].astype(float)
outdfallo['WBenUse'] = dfallo['WSWCBeneficialUse'].astype(str)
outdfallo['State'] = dfallo['State'].astype(str) 
outdfallo['AA_CFS'] = dfallo['AllocationAmount'].astype(float) 
outdfallo['AA_AF'] = dfallo['IrrigatedAcreage'].astype(float) 
outdfallo['AAM_AF'] = dfallo['AllocationMaximum'].astype(float) 
outdfallo['PD'] = dfallo['Date']
outdfallo['WSN'] = dfallo['WaterSourceName'].astype(str)

outdfallo

,AA_ID,SiteUUID,SN_ID,Lat,Long,WBenUse,State,AA_CFS,AA_AF,AAM_AF,PD,WSN
0,1370234,WA_25923,2032185.0,48.655508,-122.781905,Agricultural,WA,0.000045,2.00,2.0000,1940-01-01,Unspecified
1,1370235,WA_42838,2032207.0,48.474909,-122.309847,Agricultural,WA,0.026736,0.63,1.0741,1914-01-01,Unspecified
2,1370236,WA_38555,2032213.0,48.533434,-122.227602,Agricultural,WA,0.044560,10.92,1.0741,1895-07-01,Unspecified
3,1370237,WA_9179,2032334.0,45.674734,-121.007557,Agricultural,WA,2.673611,150.00,0.0000,1999-06-04,Unspecified
4,1370238,WA_42297,2032339.0,48.583960,-119.516129,Agricultural,WA,0.735243,20.00,0.0000,1999-07-12,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...
766354,2141585,TX_3465,C1497,30.489069,-100.023706,Agricultural,TX,0.000000,0.00,0.0000,2020-06-24,Unspecified
766355,2141586,TX_275,C3439,28.845659,-95.861930,Agricultural,TX,0.000000,0.00,0.0000,2020-06-24,Unspecified
766356,2141587,TX_4224,C1861,31.154508,-98.918007,Agricultural,TX,0.000000,0.00,0.0000,2020-06-24,Unspecified
766357,2141588,TX_4240,C3786,35.887029,-101.176992,Agricultural,TX,0.000000,0.00,0.0000,2020-06-24,Unspecified


In [17]:
#Exporting Outbound DataFrame to working csv file.
outdfallo.to_csv('P_AlloLFSite.csv', index=False)  # The output